-----
<h1><font color="#f37626">[Experiment]</font> ML-Regression 예제</h1>

- 보다 상세한 Accuinsight 파이썬 패키지 사용법은 [Accuinsight 안내 홈페이지](https://accuinsight.cloudz.co.kr/#/intro) 또는 [Accuinsight Youtube 채널](https://www.youtube.com/channel/UChFs-FAVxgG4C00h8C1MqoA)을 참조하시기 바랍니다.
- Accuinsight 패키지를 사용한 분석 코드는 [Accuinsight-github](https://github.com/AccuInsight/accuinsight_Lifecycle_example)에서 조회 가능합니다.

###  # 보스턴 주택 가격 예측
----

### 1. Import packages

In [1]:
from Accuinsight.Lifecycle.ML import accuinsight

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

2021-12-01 16:02:38.908327: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
accu = accuinsight()

### 2. Data load and split

In [3]:
boston = pd.read_csv('../data/boston_data.csv')
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [4]:
boston.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [5]:
from sklearn.model_selection import train_test_split

boston_train, boston_valid = train_test_split(boston, test_size=0.4, random_state=0)
boston_valid, boston_test = train_test_split(boston_valid, test_size=0.5, random_state=0)

y_train = boston_train.loc[:, 'MEDV']
y_valid = boston_valid.loc[:, 'MEDV']
y_test = boston_test.loc[:, 'MEDV']

X_train = boston_train.drop(['MEDV'], axis=1)
X_valid = boston_valid.drop(['MEDV'], axis=1)
X_test = boston_test.drop(['MEDV'], axis=1)

X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 303 entries, 198 to 172
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     303 non-null    float64
 1   ZN       303 non-null    float64
 2   INDUS    303 non-null    float64
 3   CHAS     303 non-null    int64  
 4   NOX      303 non-null    float64
 5   RM       303 non-null    float64
 6   AGE      303 non-null    float64
 7   DIS      303 non-null    float64
 8   RAD      303 non-null    int64  
 9   TAX      303 non-null    int64  
 10  PTRATIO  303 non-null    float64
 11  B        303 non-null    float64
 12  LSTAT    303 non-null    float64
dtypes: float64(10), int64(3)
memory usage: 33.1 KB


### 3. Build model

In [6]:
pipeline =  Pipeline([
    ('scaler', MinMaxScaler()),
    ('vote', VotingRegressor(
        estimators=[('rf', RandomForestRegressor()),
                    ('svm', SVR()),
                    ('reg', LinearRegression())
                   ])
    )
])

In [7]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('vote',
                 VotingRegressor(estimators=[('rf', RandomForestRegressor()),
                                             ('svm', SVR()),
                                             ('reg', LinearRegression())]))])

### 4. Model evaluation

In [8]:
pred_train = pipeline.predict(X_train)
rmse_train = np.sqrt(mean_squared_error(y_train, pred_train))

print(pipeline.score(X_train, y_train))
print(rmse_train)

0.8652599389393898
3.3929078154570074


In [12]:
pred_valid = pipeline.predict(X_valid)
rmse_valid = np.sqrt(mean_squared_error(y_valid, pred_valid))

print(pipeline.score(X_valid, y_valid))
print(rmse_valid)

0.7139094724639706
4.777410733611317


In [13]:
pred_test = pipeline.predict(X_test)
rmse_test = np.sqrt(mean_squared_error(y_test, pred_test))

print(pipeline.score(X_test, y_test))
print(rmse_test)

0.7810010166139085
4.327797915668433


In [14]:
# rmae
rmae_test = np.sqrt(mean_absolute_error(y_test, pred_test))
print(rmae_test)

1.6984574269346835


### 5. Run model with experiment

#### [optional] 1-(1) 메시지 설정 

message를 푸시하는 방법은 2가지 입니다.
1. 모델 학습 완료시 메시지 푸시  
    - `send_message(message = 'your_message')`  
    
    
2. 학습에 사용되는 metric이 일정 thresholds가 넘은 경우에만 메시지 푸시
    - `send_message(thresholds = 0.5)`

In [15]:
accu.send_message('[ML experiment] Boston MEDV training finished')

#### [optional] 1-(2) alarm method 설정
- web push는 기본 method이며, message가 있을 경우 자동으로 alart 됩니다.
- slack: slack channel의 hook url을 입력합니다.
- mail: mail address를 입력합니다.

In [16]:
# accu.set_slack(hook_url='hook_url')

In [17]:
# shap not supports some ensemble models, so you should set model_monitor to False
with accu.add_experiment(pipeline, X_train, y_train, X_test, y_test, model_monitor=False) as exp:
    exp.log_metrics('Rmse', rmse_test)
    exp.log_metrics('Rmae', rmae_test)

Using add_experiment(model_monitor=False)
